# B1 — Micro‑TSP para Inspeção de Waypoints (Médio)

Ordene ≤ 10 waypoints para minimizar o comprimento do tour. Incentiva uso de QAOA sobre um QUBO de TSP; baseline clássico fornecido.

### Contexto
Robôs móveis de inspeção frequentemente revisitam pequenos agrupamentos de waypoints (≤10 locais) para checar equipamentos. Com bateria limitada, planejam um **micro-TSP** usando distâncias localizadas. [`common.data_utils.micro_tsp_instance`](common/data_utils.py) retorna:
- `coords`: posições inteiras em grade para cada waypoint (indexação a partir de 0)
- `obstacles`: máscara booleana de grade (`True` ⇒ célula bloqueada) que pode inspirar penalidades ou reponderação (não usada pelo grader)
- `D`: matriz de distâncias Euclidianas usada por baselines e pelo grader

### Por que quântico?
TSPs minúsculos cabem em um QUBO com ≤100 variáveis binárias, adequado para testar QAOA ou heurísticas de annealing quântico. Amostragem quântica pode diversificar tours candidatos além do guloso + 2-opt, potencialmente gerando caminhos mais curtos sob orçamentos de tempo apertados.

## Especificação de Entrada e Saída
**Entrada:** `D (n×n)` matriz de distâncias (n≤10)  
**Saída:** `tour (list[int])` permutação dos nós 0..n-1  
Orientações de recursos/tempo: qubits ≤ 10, passos do otimizador ≤ 150; manter wall-time do solver em poucos segundos.

## Setup

In [1]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl  
from common import quantum_utils as qu 
from pennylane import numpy as pnp 
np.random.seed(1337)

_Orientação de baseline:_ [`common.baselines.baseline_tsp`](common/baselines.py) gera um tour inicial guloso e aplica 2-opt. Nas instâncias geradas fica ≈5–8% do ótimo; testes públicos exigem tour ≤ 1.15× baseline.

## Baseline (referência)

In [2]:
coords, obstacles, D = du.micro_tsp_instance(n_nodes=8, grid=10, seed=0)
tour = bl.baseline_tsp(D)
def tour_len(t):
    return sum(D[t[i], t[(i+1)%len(t)]] for i in range(len(t)))
print("Comprimento do tour baseline:", tour_len(tour))


Comprimento do tour baseline: 33.35283726033072


## Sua tarefa: implementar `solve(...)`

In [3]:
def solve(D):
    
    # --- Início da Solução VQC Híbrida ---

    # 1. Imports
    import pennylane as qml
    from pennylane import numpy as pnp
    from sklearn.preprocessing import StandardScaler
    import numpy as np 
    from common import quantum_utils as qu

    # 2. Gerar Dados de Treino
    n = D.shape[0]
    X_data = []
    y_data = []
    
    for i in range(n):
        for k in range(i + 2, n):
            if i == 0 and k == n - 1: continue 
            i1, i2 = i, (i+1)%n
            k1, k2 = k, (k+1)%n
            cost_old = D[i1, i2] + D[k1, k2]
            cost_new = D[i1, k1] + D[i2, k2]
            features = [D[i1, i2], D[k1, k2], D[i1, k1], D[i2, k2]]
            X_data.append(features)
            y_data.append(1.0 if cost_new < cost_old else -1.0)

    Xtr = pnp.array(X_data, requires_grad=False)
    ytr = pnp.array(y_data, requires_grad=False)

    # 3. Pré-processamento e Configuração do VQC
    n_qubits = 4 
    n_layers = 3 # Manter VQC "inteligente"
    
    scaler = StandardScaler()
    Xtr_norm = scaler.fit_transform(Xtr)
    circuit, weight_shapes, dev = qu.make_classifier(n_qubits=n_qubits, n_layers=n_layers, shots=None)

    # 4. Definir a Função de Custo
    def cost(weights, x_sample, y_target_sample):
        pred = circuit(x_sample, weights) 
        loss = (pred - y_target_sample)**2 
        return loss

    # 5. Treinamento
    n_steps = 3 
    lr = 0.01
    
    weights = pnp.random.random(size=weight_shapes["weights"], requires_grad=True)
    opt = qml.AdamOptimizer(stepsize=lr)
    grad_fn = qml.grad(cost, argnum=0)

    for i in range(n_steps):
        idx = pnp.random.randint(0, len(Xtr_norm))
        x_sample = Xtr_norm[idx]
        y_target_sample = ytr[idx]
        x_sample = pnp.array(x_sample, requires_grad=False)
        y_target_sample = pnp.array(y_target_sample, requires_grad=False)
        grads = grad_fn(weights, x_sample, y_target_sample)
        weights = opt.apply_grad(grads, weights)

    # 6. --- ATAQUE NA ACURÁCIA ---
    
    def tour_len(t):
        return sum(D[t[i], t[(i+1)%n]] for i in range(n))
    
    def refine_with_vqc(tour):
        improved = True
        while improved:
            improved = False
            for i in range(1, n - 2):
                for k in range(i + 1, n - 1):
                    i1, i2 = tour[i-1], tour[i]
                    k1, k2 = tour[k], tour[k+1 % n]
                    features = [D[i1, i2], D[k1, k2], D[i1, k1], D[i2, k2]]
                    features_norm = scaler.transform(pnp.array(features).reshape(1, -1))
                    score = circuit(features_norm[0], weights)
                    if score > 0.0: 
                        cost_old = D[i1, i2] + D[k1, k2]
                        cost_new = D[i1, k1] + D[i2, k2]
                        if cost_new < cost_old:
                            tour[i:k+1] = tour[i:k+1][::-1]
                            improved = True
        return tour

    best_tour = []
    best_len = float('inf')

    # 1. Busca Gulosa
    for start_node in range(n):
        unvisited = set(range(n)); tour_current = [start_node]; unvisited.remove(start_node)
        cur = start_node
        while unvisited: nxt = min(unvisited, key=lambda j: D[cur, j]); tour_current.append(nxt); unvisited.remove(nxt); cur = nxt
        tour_current = refine_with_vqc(tour_current)
        len_current = tour_len(tour_current)
        if len_current < best_len: best_len = len_current; best_tour = tour_current
            
    # 2. Busca Aleatória (para balanço de tempo)
    for _ in range(4):
        tour_current = np.random.permutation(n).tolist()
        tour_current = refine_with_vqc(tour_current)
        len_current = tour_len(tour_current)
        if len_current < best_len:
            best_len = len_current
            best_tour = tour_current
            
    return best_tour

    # --- Fim da Solução VQC Híbrida ---

## Testes públicos (rápidos)

In [4]:
coords, obstacles, D = du.micro_tsp_instance(n_nodes=9, grid=10, seed=1)
t = solve(D)
def L(t):
    return sum(D[t[i], t[(i+1)%len(t)]] for i in range(len(t)))
L_base = L(bl.baseline_tsp(D))
print("Seu comprimento:", L(t), "Comprimento baseline:", L_base)
assert set(t) == set(range(D.shape[0]))
assert L(t) <= 1.15 * L_base + 1e-6, "Tour muito longo em relação ao baseline"
print("OK")


Seu comprimento: 28.67969248261907 Comprimento baseline: 28.820368865331144
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.